In [1]:
cd ..

/home/dmitriishubin/Desktop/Thesis/HepaticVessel_experiment


In [2]:
import pandas as pd
import numpy as np
import pydicom as dicom
import nibabel as nib
import os
from sklearn.model_selection import KFold
import json

/home/dmitriishubin/anaconda3/envs/thesis/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [18]:
DATA_PATH = '../data/Task09_Spleen/'

In [19]:
img = nib.load(f'{DATA_PATH}imagesTr/spleen_10.nii.gz')
img = np.array(img.dataobj)

In [20]:
img.shape

(512, 512, 55)

In [26]:
patients = [i for i in os.listdir(f'{DATA_PATH}imagesTr/') if i.find('._spl')==-1]

In [27]:
patients

['spleen_18.nii.gz',
 'spleen_63.nii.gz',
 'spleen_16.nii.gz',
 'spleen_41.nii.gz',
 'spleen_62.nii.gz',
 'spleen_6.nii.gz',
 'spleen_47.nii.gz',
 'spleen_61.nii.gz',
 'spleen_60.nii.gz',
 'spleen_59.nii.gz',
 'spleen_22.nii.gz',
 'spleen_28.nii.gz',
 'spleen_25.nii.gz',
 'spleen_14.nii.gz',
 'spleen_3.nii.gz',
 'spleen_17.nii.gz',
 'spleen_52.nii.gz',
 'spleen_49.nii.gz',
 'spleen_56.nii.gz',
 'spleen_24.nii.gz',
 'spleen_40.nii.gz',
 'spleen_13.nii.gz',
 'spleen_45.nii.gz',
 'spleen_9.nii.gz',
 'spleen_26.nii.gz',
 'spleen_46.nii.gz',
 'spleen_38.nii.gz',
 'spleen_21.nii.gz',
 'spleen_32.nii.gz',
 'spleen_53.nii.gz',
 'spleen_12.nii.gz',
 'spleen_19.nii.gz',
 'spleen_20.nii.gz',
 'spleen_27.nii.gz',
 'spleen_29.nii.gz',
 'spleen_8.nii.gz',
 'spleen_44.nii.gz',
 'spleen_33.nii.gz',
 'spleen_31.nii.gz',
 'spleen_2.nii.gz',
 'spleen_10.nii.gz']

In [28]:
patients[0][:-7]

'spleen_18'

In [29]:
names = []

os.makedirs(f'{DATA_PATH}processed_data',exist_ok=True)

count = 0

for patient in patients:
    img = nib.load(f'{DATA_PATH}imagesTr/{patient}')
    img = np.array(img.dataobj)
    seg = nib.load(f'{DATA_PATH}labelsTr/{patient}')
    seg = np.array(seg.dataobj)
    
    patient = patient[:-7]
    
    os.makedirs(f'{DATA_PATH}processed_data/{patient}',exist_ok=True)
    
    for i in range(img.shape[-1]):
        temp_img = img[:,:,i].reshape(512, 512,1)
        temp_seg = seg[:,:,i].reshape(512, 512,1)
        
        
        np.save(f'{DATA_PATH}processed_data/{patient}/{i}.npy',temp_img)
        np.save(f'{DATA_PATH}processed_data/{patient}/{i}_seg.npy',temp_seg)
        count+=1
            

In [30]:
for index,patient in enumerate(patients):
    patients[index]=patient[:-7]

In [33]:
patients = np.array(patients)

kf = KFold(5,shuffle=True,random_state=42)
for fold,(train_index, test_index) in enumerate(kf.split(patients)):
    patients_train, patients_test = patients[train_index], patients[test_index]
    
    split = {
        'train': patients_train.tolist(),
        'val': patients_test.tolist(),
    }
    
    with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)